In [203]:
import pandas as pd
import numpy as np 
from plotly.graph_objects import *
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score


import warnings;
warnings.filterwarnings('ignore');

In [204]:

df = pd.read_csv('../datatest/AEP_hourly.csv')

In [205]:
df.head()

,Datetime,AEP_MW
0,2004-12-31 01:00:00,13478.0
1,2004-12-31 02:00:00,12865.0
2,2004-12-31 03:00:00,12577.0
3,2004-12-31 04:00:00,12517.0
4,2004-12-31 05:00:00,12670.0


In [206]:
df['Datetime']=pd.to_datetime(df['Datetime'])
df.set_index('Datetime',inplace=True)

In [207]:
df = df.resample('M').sum().reset_index()

In [208]:
df = df[:-1]
df.head()


,Datetime,AEP_MW
0,2004-10-31,10349073.0
1,2004-11-30,10677917.0
2,2004-12-31,12452864.0
3,2005-01-31,12735114.0
4,2005-02-28,11085745.0


In [209]:
iplot(Figure(data=Scatter(x =df['Datetime'],
                          y = df['AEP_MW'],)))

In [210]:
def create_configs():
    configs = list()
    trend_list = ['add','mul',None]
    s_list = ['add','mul',None]
    d_list = [True, False ]
    s_periods = 12
    
    for t in trend_list:
        for s in s_list:
            for d in d_list:
                config = dict( trend = t, seasonal = s, damped_trend = d )
                if s!=None:
                    config["seasonal_periods"] = s_periods
                configs.append(config)
    return configs
            
        
    

In [211]:
create_configs()

[{'trend': 'add',
  'seasonal': 'add',
  'damped_trend': True,
  'seasonal_periods': 12},
 {'trend': 'add',
  'seasonal': 'add',
  'damped_trend': False,
  'seasonal_periods': 12},
 {'trend': 'add',
  'seasonal': 'mul',
  'damped_trend': True,
  'seasonal_periods': 12},
 {'trend': 'add',
  'seasonal': 'mul',
  'damped_trend': False,
  'seasonal_periods': 12},
 {'trend': 'add', 'seasonal': None, 'damped_trend': True},
 {'trend': 'add', 'seasonal': None, 'damped_trend': False},
 {'trend': 'mul',
  'seasonal': 'add',
  'damped_trend': True,
  'seasonal_periods': 12},
 {'trend': 'mul',
  'seasonal': 'add',
  'damped_trend': False,
  'seasonal_periods': 12},
 {'trend': 'mul',
  'seasonal': 'mul',
  'damped_trend': True,
  'seasonal_periods': 12},
 {'trend': 'mul',
  'seasonal': 'mul',
  'damped_trend': False,
  'seasonal_periods': 12},
 {'trend': 'mul', 'seasonal': None, 'damped_trend': True},
 {'trend': 'mul', 'seasonal': None, 'damped_trend': False},
 {'trend': None,
  'seasonal': 'add',


In [228]:
def cv_run(data,config):
    mae_list = list()
    r2_list = list()
    tscv = TimeSeriesSplit(test_size = 12)
    for train_index, test_index in tscv.split(data):
        train, test = data[train_index], data[test_index]
        mod = ExponentialSmoothing(train, 
                                   **config).fit()
        predictions = mod.forecast(len(test))
        mae_list.append(mean_absolute_error(test,predictions))
        r2_list.append(r2_score(test,predictions))
    return {'mae':np.mean(mae_list),'r2':np.mean(r2_list)}

In [229]:
configs  = create_configs()

In [230]:
results = {}

In [231]:
for i in range(len(configs)):
    config = configs[i]
    try:
        results[i] = cv_run(df['AEP_MW'], config)
    except:
        results[i] = None
        continue

In [232]:
results

{0: {'mae': 441432.10962978227, 'r2': 0.6307805492415859},
 1: {'mae': 442078.46410925174, 'r2': 0.626202006804712},
 2: {'mae': 441544.00237911957, 'r2': 0.6300327970778484},
 3: {'mae': 943621.8141125534, 'r2': -2.66487636680349},
 4: {'mae': 1468007.1711752533, 'r2': -1.976758054051135},
 5: {'mae': 1192470.7471947153, 'r2': -1.0687737431493356},
 6: {'mae': 507416.78752651316, 'r2': 0.5288802950964281},
 7: {'mae': 438520.0034823834, 'r2': 0.627097796498746},
 8: {'mae': 502325.99990903615, 'r2': 0.5275219177764306},
 9: {'mae': 440406.33775860164, 'r2': 0.6284436011374099},
 10: {'mae': 1323580.3462859835, 'r2': -1.5016672227726233},
 11: {'mae': 1465008.4735898164, 'r2': -1.9784676950006428},
 12: None,
 13: {'mae': 440716.11585628137, 'r2': 0.6327851328734407},
 14: None,
 15: {'mae': 437714.9967452566, 'r2': 0.6363099124902197},
 16: None,
 17: {'mae': 834147.7961787656, 'r2': -0.072802703821126}}

In [233]:
config_opt = configs[15]

In [234]:
config_opt

{'trend': None,
 'seasonal': 'mul',
 'damped_trend': False,
 'seasonal_periods': 12}

In [236]:
train,test = df[0:-12],df[-12:]
mod = ExponentialSmoothing(train['AEP_MW'], 
                           **config_opt 
                           ).fit()
preds = mod.forecast(len(test))

In [239]:
iplot(Figure(data = [Scatter(y = test['AEP_MW'],name = 'Original'),Scatter(y = preds.values,name = 'Predictions')  ]))             